In [88]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [89]:
offense = pd.read_csv('2024 Offense.csv')
offense['Unnamed: 0'] = offense['Unnamed: 0'].shift(-2)
offense = offense.dropna()
offense = offense.drop(columns=['SH%', 'TO'])
offense['Unnamed: 0'] = offense['Unnamed: 0'].astype(str)

In [90]:
defense = pd.read_csv('2024 Defense.csv')
defense['ï»¿'] = defense['ï»¿'].shift(-2)
defense = defense.dropna()
defense = defense.drop(columns=['GB', 'P', 'PEN', 'SH', 'SOG', 'TCH', 'PAS'])
defense['CT'] = defense['CT'].astype(float)

In [91]:
goalie = pd.read_csv('2024 Goalie.csv')
goalie['Unnamed: 0'] = goalie['Unnamed: 0'].shift(-2)
goalie = goalie.dropna()
goalie = goalie.drop(columns=['SV%', 'Unnamed: 0'])

In [92]:
posession = pd.read_csv('2024 Posession.csv')

In [93]:
ct_df = defense.groupby(['GID', 'TID'])['CT'].sum().reset_index()

In [94]:
ct_df['OTID'] = ct_df.groupby('GID')['TID'].transform(lambda x: x.iloc[::-1].values)

In [95]:
clean_data = pd.merge(offense, ct_df, on=['GID', 'TID'], how='outer')

In [96]:
goalie_sa = goalie.groupby(['GID', 'TID'])['SA'].sum().reset_index()
goalie_sv = goalie.groupby(['GID', 'TID'])['SV'].sum().reset_index()
goalie = pd.merge(goalie_sa, goalie_sv, on=['GID', 'TID'], how='outer')

In [97]:
clean_data = pd.merge(clean_data, goalie, on=['GID', 'TID'], how='outer')

In [98]:
clean_data = pd.merge(clean_data, posession, on=['GID', 'TID'], how='outer')

In [99]:
cur_data = clean_data.groupby('Unnamed: 0').mean().reset_index()

In [100]:
cur_data.to_csv('cur_data.csv')

In [101]:
def make_context(df):
    to_change = [
        '1G',
        '2G',
        'A',
        'SH',
        'SOG',
        'GB',
        'TCH',
        'PAS',
        'CT',
        'SA',
        'SV',
        'PS%'
    ]

    def calculate_previous_avg(group):
        prev_avg = []
        total = 0
        count = 0
        for value in group:
            if count == 0:
                prev_avg.append(total)
            else:
                prev_avg.append(total / count)
            count += 1
            total += value
        return pd.Series(prev_avg, index = group.index)
    
    df = df.sort_values(by='GID')

    for stat in to_change:
        df[stat] = df.groupby('Unnamed: 0')[stat].apply(calculate_previous_avg).reset_index(level=0, drop=True)
    

    return df

In [102]:
clean_data = make_context(clean_data)

In [103]:
clean_data = clean_data.sort_values(by=['GID', 'TID'])
clean_data['OPS%'] = clean_data.groupby('GID')['PS%'].transform(lambda x: x.iloc[::-1].values)
clean_data['OCT'] = clean_data.groupby('GID')['CT'].transform(lambda x: x.iloc[::-1].values)
clean_data['OSA'] = clean_data.groupby('GID')['SA'].transform(lambda x: x.iloc[::-1].values)
clean_data['OSV'] = clean_data.groupby('GID')['SV'].transform(lambda x: x.iloc[::-1].values)

In [104]:
clean_data = clean_data.groupby('Unnamed: 0').apply(lambda x: x.iloc[1:]).reset_index(drop=True)

/var/folders/1j/ycj6sm297qj2d2b1b3pb8k980000gn/T/ipykernel_60218/3323778609.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  clean_data = clean_data.groupby('Unnamed: 0').apply(lambda x: x.iloc[1:]).reset_index(drop=True)


In [105]:
clean_data = clean_data.sort_values(by=['GID', 'TID'])
clean_data = clean_data[clean_data['OPS%'] != 0]

In [87]:
clean_data.to_csv('clean_data.csv')